# 1. Imports

In [1]:
import os
import numpy as np

import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report, top_k_accuracy_score

# 2. Load and Prepare Data

In [ ]:
def load_and_preprocess_images(data_folder, img_size):
    images = []
    labels = []
    label_names = sorted(os.listdir(data_folder))
    for label_id, label_name in enumerate(label_names):
        label_folder = os.path.join(data_folder, label_name)
        for filename in os.listdir(label_folder):
            img_path = os.path.join(label_folder, filename)
            img = load_img(img_path, target_size=img_size)
            img_array = img_to_array(img) / 255.0  # Normalize pixel values
            images.append(img_array)
            labels.append(label_id)
    return np.array(images), np.array(labels)

# Load and preprocess images
data_folder = "03_occupancy_grid/combined_set"
img_size = (100, 100)
images, labels = load_and_preprocess_images(data_folder, img_size)

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# 3. Create CNN Model

In [ ]:
# Define CNN architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_size[0], img_size[1], 3)),
    MaxPooling2D((2, 2)),
    Dropout(0.5),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.5),
    Flatten(),
    Dense(16, activation='relu'),
    Dense(len(np.unique(labels)), activation='softmax')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0005), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Display model summary
model.summary()

# 4. Training

In [ ]:
with tf.device('/GPU:0'):  # Use GPU 0
    # Train the model here
    history = model.fit(X_train, y_train, epochs=80, batch_size=16, validation_data=(X_test, y_test))

# 5. Evaluate

In [ ]:
# Evaluate the model on testing data
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

# Get model predictions
y_pred_probs = model.predict(X_test)  # Probability outputs
y_pred = np.argmax(y_pred_probs, axis=1)  # Convert to class labels

# Compute classification metrics
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average="macro")  # Use macro instead of weighted
top_2_acc = top_k_accuracy_score(y_test, y_pred_probs, k=2)
top_3_acc = top_k_accuracy_score(y_test, y_pred_probs, k=3)

# Display results
print(f"Test Accuracy: {accuracy:.4f}")
print(f"F1 Score (Macro): {f1:.4f}")
print(f"Top-2 Accuracy: {top_2_acc:.4f}")
print(f"Top-3 Accuracy: {top_3_acc:.4f}")

# Detailed classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# 6. Save the model

In [6]:
model.save('finetuned_model.keras')

# 7. Plot Training History

In [ ]:
import matplotlib.pyplot as plt

# Plot training history
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.title('Model accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.show()

plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()